Add reactions for glucose equivalent uptake simulation

In [5]:
import cobra as cb
import os
import pandas as pd
import csv
import sys

import settings
os.chdir(os.path.join(settings.PROJECT_ROOT,'iCBI'))

In [2]:
icbi = cb.io.read_sbml_model(os.path.join(settings.PROJECT_ROOT,'iCBI','49.SBML','iCBI676.sbml'))
isg = cb.io.load_json_model(os.path.join(settings.PROJECT_ROOT,'iSG676','iSG676_cb.json'))

# Metabolites

In [3]:
# abbreviation corresponds to bigg id with some small changes
c = pd.read_csv(os.path.join(settings.PROJECT_ROOT,'iCBI','compounds.tsv'), sep='\t')
c.head(2)

,id,abbreviation,name,formula,mass,source,inchikey,charge,is_core,is_obsolete,linked_compound,is_cofactor,deltag,deltagerr,pka,pkb,abstract_compound,comprised_of,aliases,smiles
0,cpd00001,h2o,H2O,H2O,18.0,ModelSEED,XLYOFNOQVPJJNP-UHFFFAOYSA-N,0,1,0,NaN,0,-56.687,0.50000,01:15.7,1:-1.8,NaN,NaN,NaN,O
1,cpd00002,atp,ATP,C10H13N5O13P3,504.0,ModelSEED,ZKHQWZAMYRWXGA-KQYNXXCUSA-K,-3,1,0,NaN,0,-673.850,3.04314,26:0.84;22:2.95;14:13.03;29:1.83;30:7.72,15:3.97;14:-3.66;6:-3.48;9:-9.18,NaN,NaN,NaN,O[C@@H]1[C@@H](COP(=O)(OP(=O)(OP(=O)(O)[O-])[O...


In [4]:
def ms_to_bigg1(metid):
    metid = metid.replace('-','__')
    return metid
def ms_to_bigg2(metid):
    metid = metid.replace('-','_')
    return metid

c['abbreviation1'] = c['abbreviation'].map(ms_to_bigg1)
c['abbreviation2'] = c['abbreviation'].map(ms_to_bigg2)
bigg2ms1 = {r.abbreviation1:r.id for i, r in c.iterrows()}
bigg2ms2 = {r.abbreviation2:r.id for i, r in c.iterrows()}

In [5]:
not_matched = []
bigg2ms = {}
for met in isg.metabolites:
    mid = met.id
    comp = mid[-2:]
    mid_nc = mid[:-2]
    
    if mid_nc in bigg2ms1:
        bigg2ms[mid_nc] = bigg2ms1[mid_nc]
    elif mid_nc in bigg2ms2:
        bigg2ms[mid_nc] = bigg2ms2[mid_nc]
    else:
        not_matched.append(mid_nc)
    
len(not_matched)

239

In [6]:
df = pd.DataFrame({'bigg':list(bigg2ms.keys()), 'ms':list(bigg2ms.values())})
df.head()

,bigg,ms
0,h2o,cpd00001
1,atp,cpd00002
2,nad,cpd00003
3,nadh,cpd00004
4,nadph,cpd00005


In [7]:
df.to_csv('bigg2ms_met_raw.csv',index=False) # Manual curation will be performed to add relevant unmatched metabolites

In [8]:
# show unmatched
columnize(not_matched)

'orn__L        dacephc        sel          cell5          escys          cell5\ndocylp        dd2coa         selhcys      cell3          adenSCP        cell4\ncmpacna       6doer          selcyst      acnamp         3hohodcoa      cell3\nfdxr_42       airb           phser        slac           3osteACP       glceq\nfdxo_42       neunng         methsel      3hpimcoa       toctd2eACP     btd_M\nham           cmpglna        cyanh_gen    6cahexecoa     cproc          ppoh \nudpacgal      am6ohd         cyang_gen    6hch1eccoa     cproccoa       ppoh \ncdpdag_gen    pgp_gen        3ohexACP     2a3o4pobut     35cdgmp        3mboh\nquin          3g12dgr_gen    3hhexACP     ribprn         5m24decoa      3mboh\nrdxr          5a4idca        thex2eACP    thfyglu        5ho5mhe4ecoa \npglyc_gen     4c5h2od        3ooctACP     hmpscoa        na15dap      \nsincoa        3hmoa          toct2eACP    bzylsuccoa     6txan5mp     \nrdxo          but2eACP       3odecACP     2dhc1ccoa      6tgsnmp      \n

# Reactions

In [9]:
def nfix(name):
    # removes differences in icbi naming
    name = name.replace('_c0','')
    name = name.replace('_e0','')
    return name

rxnid = [nfix(r.id) for r in icbi.reactions]

# Load curated map 
ms2bigg_met = {}
with open(os.path.join(settings.PROJECT_ROOT, 'iCBI', 'bigg2ms_met.csv'), 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        ms2bigg_met[row['ms']] = row['bigg']
        
def ex_remap(rid):
    # converts exchange reaction of the form EX_cpd00001_e0 into EX_h20_e
    met_id = rid[3:-3]
    if met_id in ms2bigg_met:
        return 'EX_{}_e'.format(ms2bigg_met[met_id])
    else:
        return rid
    

In [10]:
bigg2ms_rxn = {}
unmatched_ms = []
for rxn in icbi.reactions:
    if rxn.id.startswith('EX_'):
        biggid = ex_remap(rxn.id)
    else:
        biggid = nfix(rxn.id)
    try:
        r_isg = isg.reactions.get_by_id(biggid)
        bigg2ms_rxn[r_isg.id] = rxn.id
    except KeyError:
        unmatched_ms.append(rxn.id)
len(unmatched_ms) 

19

In [11]:
df = pd.DataFrame({'bigg':list(bigg2ms_rxn.keys()), 'ms':list(bigg2ms_rxn.values())})
df.to_csv('bigg2ms_rxn_raw.csv',index=False) # Manual curation will be performed to add relevant unmatched metabolites
df.head()

,bigg,ms
0,PPA,PPA_c0
1,ACLS,ACLS_c0
2,PDHam1hi,PDHam1hi_c0
3,ECH,ECH_c0
4,AHEXASE3,AHEXASE3_c0


In [12]:
columnize(unmatched_ms)

'PFK_3_ppi_c0    EX_3mboh_e_e0   EX_cit_e_e0  EX_cpd02701_c0  EX_cpd03719_e0\nEX_na1_e_e0     EX_fuc__L_e_e0  EX_g1p_e_e0  EX_cpd03721_e0  EX_cpd01949_e0\nEX_ppoh_e_e0    EX_glc__D_e_e0  FRDx_c0      EX_cpd01376_e0  EX_cpd11416_c0\nEX_2mbtoh_e_e0  EX_fru_e_e0     bio1         EX_cpd03720_e0\n'